In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [109]:
data = pd.read_csv('Data.txt')
data.head()

,7.74108464
0,7.762000
1,7.788000
2,7.808400
3,7.823564
4,7.812000


In [110]:
# Rename a column

data.rename(columns={'7.74108464':'Seq_num'},inplace=True)
data

,Seq_num
0,7.762000
1,7.788000
2,7.808400
3,7.823564
4,7.812000
...,...
4994,7.738000
4995,7.738000
4996,7.740000
4997,7.739548


In [161]:
# print the data now.
data.tail(10)

,Seq_num
4989,7.730600
4990,7.734000
4991,7.738000
4992,7.740793
4993,7.738000
4994,7.738000
4995,7.738000
4996,7.740000
4997,7.739548
4998,7.742000


In [112]:
data.shape

(4999, 1)

In [113]:
import tensorflow 

In [114]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.layers import Flatten
import math
from sklearn.metrics import mean_squared_error



In [115]:
# prepairing independent and dependent data

def prepare_data(time_ser_data,n_features):
    x,y=[],[]
    for i in range(len(time_ser_data)):
        # find the end of this pattern

        end_pattern = i+n_features
        # check if we are beyond the sequenec
        if end_pattern >len(time_ser_data)-1:
            break

        # gather input and output part of the pattern
        seq_x,seq_y = time_ser_data[i:end_pattern],time_ser_data[end_pattern]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x),np.array(y)


In [116]:
# define input sequence

time_ser_data =data.iloc[:,0]
time_ser_data.shape

(4999,)

In [167]:
# choose a  number of time steps

n_steps= 4

# split into samples
x , y = prepare_data(time_ser_data,n_steps)

In [168]:
print(x) 
x.shape

[[7.762      7.788      7.8084     7.82356438]
 [7.788      7.8084     7.82356438 7.812     ]
 [7.8084     7.82356438 7.812      7.82      ]
 ...
 [7.74079268 7.738      7.738      7.738     ]
 [7.738      7.738      7.738      7.74      ]
 [7.738      7.738      7.74       7.73954768]]


(4995, 4)

In [169]:
print(y)
y.shape

[7.812      7.82       7.8288     ... 7.74       7.73954768 7.742     ]


(4995,)

In [170]:
# convert sample feature [samples, timesteps] to 3D [samples,timesteps,n_features]

n_feature = 1
x = x.reshape((x.shape[0],x.shape[1],n_feature))
x.shape 

(4995, 4, 1)

In [171]:
x.shape[0] , x.shape[1]

(4995, 4)

In [172]:
# implement LSTM model.

model = Sequential()
model.add(LSTM(50,activation='relu',return_sequences=True, input_shape=(n_steps,n_feature )))
model.add(LSTM(50,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss = 'mse')

In [173]:
# fit model
#  number of passes of the entire training dataset 

model.fit(x,y ,epochs=100 , verbose=1)

Epoch 1/100
157/157 [==============================] - 3s 7ms/step - loss: 5.9710
Epoch 2/100
157/157 [==============================] - 1s 7ms/step - loss: 0.0717
Epoch 3/100
157/157 [==============================] - 1s 7ms/step - loss: 0.0538
Epoch 4/100
157/157 [==============================] - 1s 6ms/step - loss: 0.0371
Epoch 5/100
157/157 [==============================] - 1s 6ms/step - loss: 0.0213
Epoch 6/100
157/157 [==============================] - 1s 6ms/step - loss: 0.0080
Epoch 7/100
157/157 [==============================] - 1s 6ms/step - loss: 9.8401e-04
Epoch 8/100
157/157 [==============================] - 1s 6ms/step - loss: 4.6581e-04
Epoch 9/100
157/157 [==============================] - 1s 6ms/step - loss: 4.2006e-04
Epoch 10/100
157/157 [==============================] - 1s 6ms/step - loss: 4.7433e-04
Epoch 11/100
157/157 [==============================] - 1s 6ms/step - loss: 4.1867e-04
Epoch 12/100
157/157 [==============================] - 1s 7ms/step - loss: 

In [174]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 4, 50)             10400     
                                                                 
 lstm_12 (LSTM)              (None, 50)                20200     
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [176]:
# predict next 4 days

x_input= np.array([7.730600,7.734000,7.738000,7.740793])

list_x= list(x_input)
lst_output = []
i = 0
while(i<4):
    if(len(list_x)>4):
        x_input=np.array(list_x[1:])
        print('{} day input {}'.format(i,x_input))

        # print x_input
        x_input= x_input.reshape((1,n_steps,n_feature))

        # print y_pred

        y_pred = model.predict(x_input,verbose = 0)
        print('{} day output {}'.format(i,y_pred))
        list_x.append(y_pred[0][0])
        list_x = list_x[1:]

        lst_output.append(y_pred[0][0])
        i = i+1
    else:
        x_input= x_input.reshape((1,n_steps,n_feature))
        y_pred = model.predict(x_input,verbose = 0)
        print(y_pred[0])
        list_x.append(y_pred[0][0])
        lst_output.append(y_pred[0][0])
        i = i+1
    print(lst_output)
        


    


[7.739171]
[7.739171]
1 day input [7.734      7.738      7.740793   7.73917103]
1 day output [[7.7413306]]
[7.739171, 7.7413306]
2 day input [7.738      7.740793   7.73917103 7.74133062]
2 day output [[7.7425146]]
[7.739171, 7.7413306, 7.7425146]
3 day input [7.740793   7.73917103 7.74133062 7.74251461]
3 day output [[7.743999]]
[7.739171, 7.7413306, 7.7425146, 7.743999]
